In [2]:
import pandas as pd # pandas hilft, Daten in tabellarischer Form besser zu verarbeiten

In [3]:
import spacy

In [4]:
from spacy.matcher import Matcher

In [5]:
nlp = spacy.load("de_core_news_sm")

In [13]:
# um später 500 Reden verarbeiten zu können, muss die Speicherkapazität erhöht werden

nlp.max_length = 1500000

In [6]:
with open ("speeches.csv", "r", encoding="UTF-8") as f:
    text = f.read()

In [7]:
opendiscourse = pd.read_csv("speeches.csv")

In [8]:
print(len(opendiscourse))

907644


In [9]:
# alle Reden der AfD extrahieren, um nur mit diesen weiter zu arbeiten:

afd = opendiscourse.query('factionId == 0')
print(afd)

             id  session  electoralTerm            firstName      lastName  \
846693  1000004        1             19                Bernd       Baumann   
846728  1000038        2             19                Bernd       Baumann   
846738  1000050        2             19             Jan Ralf         Nolte   
846757  1000068        2             19              Norbert  Kleinwächter   
846780  1000091        2             19  Eberhardt Alexander       Gauland   
...         ...      ...            ...                  ...           ...   
907560  1060872      228             19              Rüdiger      Lucassen   
907589  1060902      228             19                 Götz      Frömming   
907592  1060904      228             19                 Götz      Frömming   
907610  1060923      228             19                Joana         Cotar   
907629  1060943      228             19            Gottfried         Curio   

        politicianId                                      speec

In [10]:
print(len(afd))

4437


In [11]:
# Index der nur-AfD-Reden resetten:

afd.reset_index(drop=True)

,id,session,electoralTerm,firstName,lastName,politicianId,speechContent,factionId,documentUrl,positionShort,positionLong,date
0,1000004,1,19,Bernd,Baumann,11004662,\n\nHerr Präsident! Meine Damen und Herren! Im...,0,https://dip21.bundestag.de/dip21/btp/19/19001.pdf,Member of Parliament,NaN,2017-10-24
1,1000038,2,19,Bernd,Baumann,11004662,\n\nHerr Präsident! Meine Damen und Herren! Wi...,0,https://dip21.bundestag.de/dip21/btp/19/19002.pdf,Member of Parliament,NaN,2017-11-21
2,1000050,2,19,Jan Ralf,Nolte,11004842,\n\nSehr geehrter Herr Präsident! Sehr geehrte...,0,https://dip21.bundestag.de/dip21/btp/19/19002.pdf,Member of Parliament,NaN,2017-11-21
3,1000068,2,19,Norbert,Kleinwächter,11004781,\n\nHerr Präsident! Werte Kollegen! Dieser Ein...,0,https://dip21.bundestag.de/dip21/btp/19/19002.pdf,Member of Parliament,NaN,2017-11-21
4,1000091,2,19,Eberhardt Alexander,Gauland,11004724,\n\nHerr Präsident! Meine Damen und Herren!\n\...,0,https://dip21.bundestag.de/dip21/btp/19/19002.pdf,Member of Parliament,NaN,2017-11-21
...,...,...,...,...,...,...,...,...,...,...,...,...
4432,1060872,228,19,Rüdiger,Lucassen,11004807,\n\nFrau Präsidentin! Meine Damen und Herren! ...,0,https://dip21.bundestag.de/dip21/btp/19/19228.pdf,Member of Parliament,NaN,2021-05-07
4433,1060902,228,19,Götz,Frömming,11004722,\n\nSehr geehrte Frau Präsidentin! Sehr geehrt...,0,https://dip21.bundestag.de/dip21/btp/19/19228.pdf,Member of Parliament,NaN,2021-05-07
4434,1060904,228,19,Götz,Frömming,11004722,"\n\nWir meinen, auch hier sollten wir aus den ...",0,https://dip21.bundestag.de/dip21/btp/19/19228.pdf,Member of Parliament,NaN,2021-05-07
4435,1060923,228,19,Joana,Cotar,11004696,\n\nFrau Präsidentin! Werte Kollegen! Es steht...,0,https://dip21.bundestag.de/dip21/btp/19/19228.pdf,Member of Parliament,NaN,2021-05-07


In [14]:
# damit spaCy mit der Tabelle arbeiten kann, müssen die Texte der Reden als spaCy-doc extrahiert werden
# weil die Datenmenge so groß ist, verwenden wir nur die letzten 500 Zeilen (= die aktuellsten Reden)
               
reden = " ".join(afd.tail(500)["speechContent"].astype(str))

doc = nlp(reden)

print(len(doc))

246180


In [15]:
# nach Bestandteilen von Wörtern suchen wir mit Regex, da der Matcher nicht innerhalb von Tokens suchen kann

import re

In [34]:
pattern1 = r"(Volk|Völker\w*)\b"

matches = re.findall(pattern1, reden)

print(matches)

['Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Völker', 'Völkermord', 'Völker', 'Völkerschaften', 'Völkerschaften', 'Völker', 'Völker', 'Völkern', 'Volk', 'Völkerbund', 'Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Völker', 'Völkermord', 'Völkermord', 'Völkermords', 'Völkermords', 'Völker', 'Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Volk', 'Völkern', 'Völker', 'Völker', 'Volk', 'Völker', 'Volk', 'Volk', 'Volk', 'Volk', 'Völkerrecht', 'Völkerrecht', 'Volk', 'Volk', 'Völker', 'Völker', 'Volk', 'Völkermord', 'Völkermord', 'Völkermord', 'Völkermord', 'Völkermord', 'Völkermord', 'Völkermords', 'Volk', 'Völkerschaften', 'Volk', 'Volk']


In [24]:
pattern2 = r"(\wogenannt\w+ [a-z]+ [A-Z]\w+)|(\wogenannt\w+ [A-Z]\w+)"

matches = re.findall(pattern2, reden)

print(matches)

[('', 'sogenannten Kinderrechte'), ('sogenannten gesunden Menschenverstand', ''), ('sogenannten schwarzen Liste', ''), ('', 'sogenannten Klimarettungsmaßnahmen'), ('', 'sogenannte Energiewende'), ('', 'sogenanntes Baulandmobilisierungsgesetz'), ('', 'sogenannte Schutzmasken'), ('', 'sogenannten Great'), ('', 'sogenannten Lockdown'), ('', 'sogenannte Energiewende'), ('sogenannten instabilen Erneuerbaren', ''), ('', 'sogenannte Energiewende'), ('', 'sogenannte Recht'), ('', 'sogenannten Vogel'), ('sogenannten regelbasierten Multilateralität', ''), ('', 'sogenannten Kollateralschaden'), ('sogenannten symptomlos Erkrankten', ''), ('sogenannte echte Doppelname', ''), ('sogenannte amerikanische Mini', ''), ('sogenannte disziplinierte Demokratie', ''), ('', 'sogenannte Neiddebatte'), ('', 'sogenannte Kolonialmandat'), ('sogenannten begrenzten Einzelermächtigung', ''), ('', 'sogenannte Europäische'), ('', 'sogenannten Globalen'), ('', 'sogenannten Eigenmittelbeschluss'), ('', 'sogenannter Flüc